In [3]:
import pandas as pd
from functions import *

disable_progress = True  # Set to False to enable progress bar

# Get the paths for input data and output directory
data_path, output_path = get_paths()

# Define parameter ranges for each denoiser
GAUSSIAN_SIGMAS = [2.5, 3, 3.5]
MEDIAN_SIZES = [5, 8, 10]

def main():
    """Main script to evaluate denoisers and save results."""
    all_results = []

    # Choose which denoiser to use:
    denoiser_name = "Gaussian"  # "Median" to use the Median filter

    # Select the appropriate denoiser function and parameters
    denoiser, denoiser_params = select_denoiser(denoiser_name)

    # Process images with the selected denoiser
    if denoiser_name == "Gaussian":
        for sigma in tqdm(GAUSSIAN_SIGMAS, disable=disable_progress):
            # Update parameters dynamically for Gaussian filter
            denoiser_params = {'sigma': sigma}

            # Process images and append results
            denoiser_results = process_images(data_path, num_images=2, denoiser=denoiser, disable_progress=disable_progress, **denoiser_params)
            for result in denoiser_results:
                result.extend([denoiser_name, f"sigma={sigma}"])
            all_results.extend(denoiser_results)

        result_filename = "Gaussian_denoiser_results.csv"  # Save as Gaussian-specific results

    elif denoiser_name == "Median":
        for size in MEDIAN_SIZES:
            # Update parameters dynamically for Median filter
            denoiser_params = {'size': size}

            # Process images and append results
            denoiser_results = process_images(data_path, num_images=2, denoiser=denoiser, disable_progress=disable_progress, **denoiser_params)
            for result in denoiser_results:
                result.extend([denoiser_name, f"size={size}"])
            all_results.extend(denoiser_results)

        result_filename = "Median_denoiser_results.csv"  # Save as Median-specific results

    # Convert results to a DataFrame and reorder columns
    all_results_df = pd.DataFrame(all_results, columns=[
        'ImageIndex', 'Channel', 'PSNR', 'SI-PSNR', 'SSIM', 'Runtime', 'RAM Usage', 'DenoiserType', 'Parameter'
    ])
    all_results_df = all_results_df[['DenoiserType', 'Parameter', 'ImageIndex', 'Channel', 
                                     'PSNR', 'SI-PSNR', 'SSIM', 'Runtime', 'RAM Usage']]

    # Display and save all results
    display_styled_results(all_results_df, output_path, result_filename, title=f"{denoiser_name} Denoiser Results")

    # Compute averages and save
    avg_results = compute_averages(all_results_df)
    display_styled_results(avg_results, output_path, f"average_{denoiser_name}_denoiser_results.csv", title=f"Average {denoiser_name} Results")

if __name__ == "__main__":
    main()



Gaussian Denoiser Results:


,DenoiserType,Parameter,ImageIndex,Channel,PSNR,SI-PSNR,SSIM,Runtime,RAM Usage
0,Gaussian,sigma=2.5,001,0,30.41,31.81,0.8954,0.0043 s,5.30 MB
1,Gaussian,sigma=2.5,001,1,31.26,31.54,0.8090,0.0033 s,1.34 MB
2,Gaussian,sigma=2.5,001,2,31.81,32.09,0.7988,0.0046 s,0.08 MB
3,Gaussian,sigma=2.5,002,0,38.57,38.67,0.9782,0.0219 s,1.06 MB
4,Gaussian,sigma=2.5,002,1,38.16,41.47,0.9783,0.0042 s,0.00 MB
5,Gaussian,sigma=2.5,002,2,38.08,40.05,0.9649,0.0050 s,0.66 MB
6,Gaussian,sigma=3,001,0,30.92,31.18,0.8967,0.0038 s,0.06 MB
7,Gaussian,sigma=3,001,1,30.44,31.60,0.8126,0.0038 s,1.03 MB
8,Gaussian,sigma=3,001,2,32.33,33.05,0.8370,0.0038 s,0.00 MB
9,Gaussian,sigma=3,002,0,34.83,37.59,0.9776,0.0036 s,0.75 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/Gaussian_denoiser_results.csv

Average Gaussian Results:


,DenoiserType,Parameter,Channel,PSNR,SI-PSNR,SSIM,Runtime,RAM Usage
0,Gaussian,sigma=2.5,0,34.49,35.24,0.9368,0.0131 s,3.18 MB
1,Gaussian,sigma=2.5,1,34.71,36.51,0.8936,0.0037 s,0.67 MB
2,Gaussian,sigma=2.5,2,34.95,36.07,0.8819,0.0048 s,0.37 MB
3,Gaussian,sigma=3,0,32.87,34.38,0.9371,0.0037 s,0.41 MB
4,Gaussian,sigma=3,1,34.88,36.02,0.8938,0.0037 s,0.83 MB
5,Gaussian,sigma=3,2,36.06,36.46,0.9029,0.0038 s,0.26 MB
6,Gaussian,sigma=3.5,0,32.76,33.30,0.9268,0.0042 s,0.45 MB
7,Gaussian,sigma=3.5,1,34.91,35.55,0.8896,0.0044 s,0.59 MB
8,Gaussian,sigma=3.5,2,34.80,36.28,0.9130,0.0041 s,0.26 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/average_Gaussian_denoiser_results.csv
